Anggota :
- Paizal Merdijaya
- Alviatul Nazila
- Clarissa Sanindita Reikisyifa

#Pengenalan

##Latar Belakang

Pada era digital saat ini, layanan transportasi berbasis aplikasi telah menjadi bagian integral dari kehidupan sehari-hari banyak orang. Layanan seperti Lyft, Uber, dan lainnya memungkinkan konsumen untuk memesan tumpangan dengan mudah melalui ponsel mereka, menawarkan kenyamanan dan efisiensi yang tak tertandingi oleh moda transportasi tradisional.

Salah satu aspek paling menarik dari layanan ini adalah penentuan harga. Banyak dari platform ini memanfaatkan algoritma harga dinamis yang menyesuaikan biaya tumpangan berdasarkan berbagai faktor, seperti permintaan, waktu, cuaca, dan banyak lagi. Untuk perusahaan transportasi, pemahaman mendalam tentang dinamika harga ini adalah kunci untuk meningkatkan pendapatan dan efisiensi operasional.

Dalam konteks ini, kami memilih untuk menganalisis dataset yang berkaitan dengan perjalanan yang dipesan melalui layanan Lyft di kota tertentu. Analisis data ini dapat memberikan wawasan berharga tentang bagaimana faktor-faktor tertentu mempengaruhi harga, serta bagaimana perusahaan bisa mengoptimalkan strategi penentuan harga mereka.

##Deskripsi Data

Dataset yang digunakan dalam penelitian ini berisi informasi tentang perjalanan yang dipesan melalui layanan Lyft. Spesifikasi data sebagai berikut:

Jumlah Data: 6 entri (sampel)
- Kelas: Terdiri dari berbagai jenis layanan Lyft seperti 'Shared', 'Lux', 'Lyft', 'Lux Black XL', 'Lyft XL', dan 'Lux Black'.

- Sumber Data: Data ini diambil dari kaggle

- Link Data : https://www.kaggle.com/datasets/brllrb/uber-and-lyft-dataset-boston-ma


Beberapa kolom penting dalam dataset ini meliputi:
- timestamp, hour, day, month, datetime: informasi waktu pemesanan.
source, destination: lokasi awal dan tujuan perjalanan.

- cab_type, product_id, name: informasi tentang jenis layanan Lyft.
price: harga tumpangan.
distance: jarak perjalanan

-surge_multiplier: pengali tarif yang digunakan saat permintaan tinggi.
temperature, apparentTemperature, humidity, windSpeed, dll.: data cuaca pada saat pemesanan.

##Tujuan Penelitian

Tujuan dari penelitian ini adalah:

- Memahami dinamika penentuan harga dalam layanan transportasi berbasis aplikasi seperti Lyft.

- Menganalisis bagaimana berbagai faktor, terutama kondisi cuaca, mempengaruhi harga tumpangan.

- Memberikan rekomendasi kepada perusahaan transportasi tentang bagaimana mengoptimalkan strategi penentuan harga mereka berdasarkan analisis data.

Dengan memahami hubungan antara faktor-faktor ini dan harga, kita dapat memberikan wawasan berharga yang dapat membantu perusahaan transportasi dalam mengambil keputusan bisnis yang lebih tepat dan efisien

#Import Library

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer


#Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
url = '/content/drive/MyDrive/dataset/final_project/rideshare_kaggle.csv'
df = pd.read_csv(url)

In [ ]:
df.head()

#Data Cleaning

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df['price'].fillna(df['price'].mean(), inplace=True)

#Exploration Data

In [ ]:
df.describe()

Dengan data statistik yang Anda berikan, kita dapat menyimpulkan informasi berikut:

- Jumlah Data: Ada sekitar 693,071 entri dalam dataset.

- Timestamp: Rentang waktu data mencakup sekitar 1.543204×1091.543204×109 hingga 1.545161×1091.545161×109, yang menggambarkan timestamp dalam format Unix

- Hour, Day, Month: Waktu pemesanan berkisar dari jam 0 hingga 23. Tanggal pemesanan mencakup tanggal 1 hingga 30. Data berasal dari bulan 11 dan 12.

- Price (Harga): Harga rata-rata perjalanan adalah $16.55, harga berkisar dari $2.50 hingga $97.50.

- Distance (Jarak): Jarak rata-rata perjalanan adalah sekitar 2.19 mil. Jarak perjalanan berkisar dari 0.02

- dsb

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(df["price"], bins=20, color="skyblue", edgecolor="black")
plt.title("Distribusi Harga")
plt.xlabel("Harga ($)")
plt.ylabel("Frekuensi")

plt.subplot(1, 2, 2)
plt.hist(df["temperature"], bins=20, color="salmon", edgecolor="black")
plt.title("Distribusi Suhu")
plt.xlabel("Suhu (°F)")
plt.ylabel("Frekuensi")

plt.tight_layout()
plt.show()

1. Distribusi Harga:
   - Mayoritas harga tampaknya berkumpul di kisaran yang lebih rendah, menunjukkan bahwa banyak tumpangan memiliki biaya yang relatif lebih rendah
   - Ada beberapa puncak atau modus pada distribusi, menunjukkan beberapa titik harga yang paling umum ada pada rentang harga 20$

2. Distribusi Suhu:
   - Suhu tampaknya memiliki distribusi yang lebih merata daripada harga.
   - Ada beberapa kisaran suhu di mana frekuensinya sedikit lebih tinggi, menunjukkan bahwa beberapa suhu lebih umum selama periode waktu yang dianalisis pada rentang 40°F - 45°F.

In [ ]:
grouped = df.groupby("name")["price"].mean().sort_values(ascending=False)
grouped

Dapat dilihat bahwa "Lux Black XL" memiliki harga rata-rata tertinggi, sedangkan "Shared" memiliki harga rata-rata terendah.

In [ ]:
plt.figure(figsize=(10, 6))
grouped.plot(kind="bar", color="teal")
plt.title("Harga Rata-rata berdasarkan Jenis Layanan Lyft")
plt.xlabel("Jenis Layanan")
plt.ylabel("Harga Rata-rata ($)")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

Dari visualisasi di atas, kita dapat dengan jelas melihat perbedaan harga rata-rata berdasarkan jenis layanan Lyft:
- Layanan premium seperti Lux Black XL dan Black SUV memiliki harga rata-rata yang jauh lebih tinggi dibandingkan dengan layanan lainnya.
- Sebagai layanan bersama, Shared memiliki harga rata-rata terendah, yang sesuai dengan ekspektasi karena layanan tersebut memungkinkan penumpang untuk membagi biaya tumpangan dengan orang lain.

#Data Preprocessing

In [ ]:
X = df['distance']
y = df['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.values.reshape(-1, 1)
X_test = X_test.values.reshape(-1, 1)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Pendefisian Model

In [ ]:
model = LinearRegression()

#Pelatihan Model

In [ ]:
model.fit(X_train_scaled, y_train)

#Evaluasi Model

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
# Evaluasi kinerja model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R^2): {r2}')

In [ ]:
model.score(x_train, y_train)

#Model Inference